# 模型评价：分类模型的常用评价指标

#### 基本指标：误差率

指标解释：错分类样本占总样本的比例

#### 基本指标：准确率

指标解释：正确分类样本占总样本的比例

指标解读：准确率越接近1，模型越准确

#### 混淆矩阵

| 真实情况 | 预测：正例   | 预测：反例   |
| -------- | ------------ | ------------ |
| 正例     | TP（真正例） | FN（假反例） |
| 反例     | FP（假正例） | TN（真反例） |

#### 衍生指标：查准率（precision）

指标解释：所有真正例占所有预测为正的样本的比例

指标举例：在商品推荐的过程中，我们会关心所有推荐给用户的商品（预测为正）中有多少是客户真正喜欢的（真正例）

#### 衍生指标：查全率（recall）

指标解释：所有真正例占所有真实为正的样本的比例

指标举例：在银行用户风险识别中，我们会关心，所有有风险的用户，有多少能被我们的模型识别出来

#### 其他指标：ROC曲线与AUC值

ROC曲线：以真正例比率为纵轴、假正例率为横轴，采用不同的截断点，来绘制ROC曲线

AUC值：ROC曲线与坐标轴构成的图形面积

指标解读：auc指标越接近1，则代表模型准确率越高，auc值等于0.5，代表模型准确率与随机猜测准确率一致，auc值小于0.5：模型效果不如随机猜测

In [ ]:
# 使用sklearn查看回归模型的各项指标
import pandas as pd
import matplotlib.pyplot as plt
# 样例数据读取
df = pd.read_excel('realestate_sample_preprocessed.xlsx')
# 根据共线性矩阵，保留与房价相关性最高的日间人口，将夜间人口和20-39岁夜间人口进行比例处理
def age_percent(row):
    if row['nightpop'] == 0:
        return 0
    else:
        return row['night20-39']/row['nightpop']
df['per_a20_39'] = df.apply(age_percent,axis=1)
df = df.drop(columns=['nightpop','night20-39'])
# 制作标签变量
price_median = df['average_price'].median()
print(price_median)
df['is_high'] = df['average_price'].map(lambda x: True if x>= price_median else False)
print(df['is_high'].value_counts())
# 数据集基本情况查看
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())

In [ ]:
# 划分数据集
x = df[['complete_year','area', 'daypop', 'sub_kde',
       'bus_kde', 'kind_kde','per_a20_39']]
y = df['is_high']
print(x.shape)
print(y.shape)

In [ ]:
# 建立分类模型
# 使用pipeline整合数据处理、特征筛选与模型
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
# 构建模型工作流
pipe_clf = Pipeline([
        ('sc',StandardScaler()),
        ('power_trans',PowerTransformer()),
        ('polynom_trans',PolynomialFeatures(degree=3)),
        ('lgostic_clf', LogisticRegression(penalty='l1', fit_intercept=True, solver='liblinear'))
        ])
print(pipe_clf)

In [ ]:
# 查看模型表现
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
warnings.filterwarnings('ignore')
pipe_clf.fit(x,y)
y_predict = pipe_clf.predict(x)
print(f'accuracy score is: {accuracy_score(y,y_predict)}')
print(f'precision score is: {precision_score(y,y_predict)}')
print(f'recall score is: {recall_score(y,y_predict)}')
print(f'auc: {roc_auc_score(y,y_predict)}')

## 谢谢大家观看